In [1]:
%matplotlib notebook
import sys
import pandas as pd

#sys.path.insert(0,'/global/project/projectdirs/metatlas/anaconda/lib/python2.7/site-packages' )
import qgrid

from metatlas import metatlas_objects as metob

df = pd.read_csv('20151130_Atlas_POS_HILIC_LS_Validated.csv',sep = ',')

df.columns = [x.lower() for x in df.columns]
qgrid.show_grid(df, precision=5)

# TODO: Make one Atlas.  
# TODO: Add column labels for each mzref, rtref, and fragmentation_ref
# TODO: Make template for this
# TODO: Make error checker to ensure Adduct is conforming to specification
# TODO: make error checker to ensure modification is conforming to specification

In [13]:
for c in metob.retrieve('Atlas',name='%_LS_%', username='*'):
    print c.name
# 2-acetamido-2-deoxy-beta-D-glucopyranosyl-(1->4)-2-amino-2-deoxy-D-glucopyranose Cite this Record


20151130_LS_Negative_Hilic_QExactive_Archetypes
20151130_LS_Positive_Hilic_QExactive_Archetypes_ISTDs
20151130_LS_Negative_Hilic_QExactive_Archetypes_ISTDs
20151130_LS_Positive_Hilic_QExactive_Archetypes
20151130_LS_Positive_Hilic_QExactive_Archetypes


In [3]:

# compounds that have the wrong compound name will be listed
# Keep running this until no more compounds are listed
for x in df.index:
    if not metob.retrieve('Compounds',name=df.name[x]):
        print df.name[x], "is not in database"

agmatine is not in database
L-glucose is not in database


In [11]:
df.to_csv('saveit.tab', sep='\t')

In [12]:
myCompounds = []
rt_min = []
rt_max = []
rt_peak = []
mz = []
mz_tolerance = []
for x in df.index:
    c = metob.retrieve('Compounds',name=df.name[x])[0] #currently, all copies of the molecule are returned.  The 0 is the most recent one. 
    #TODO: Consider possible change to make thinkgs like compound should be immutable and not behave this way
    myCompounds.append(c)
    rt_min.append(df.rt_min[x])
    rt_max.append(df.rt_max[x])
    rt_peak.append(df.rt_peak[x])
    mz.append(df.mz[x])
#     mz.append(c.MonoIsotopic_molecular_weight - 1.007276)

    mz_tolerance.append(df.mz_threshold[x])
    dmz = 1e6 * (c.mono_isotopic_molecular_weight + 1.007276 - df.mz[x] ) / ( df.mz[x])
#     if abs(dmz) > 100:
#         dmz = 1e6 * (c.MonoIsotopic_molecular_weight + 59.013851 - df.mz[x] ) / df.mz[x]
#         print c.MonoIsotopic_molecular_weight - df.mz[x]
        
    print '%5.2f'%abs(dmz), "\t", c.name,c.formula,c.mono_isotopic_molecular_weight,c.description, df.rt_min[x], df.rt_max[x], df.mz[x]#, c.InChI  
    #TODO: See above todo's.  this block needs to be rebuilt

56.05 	L-alanine C3H7NO2 89.0476760864  10.52 11.52 90.06
 2.87 	ectoine C6H10N2O2 142.074234009  9.41 10.41 143.0811
 0.36 	DL-methionine sulfoxide C5H11NO3S 165.045964212 http://pubchem.ncbi.nlm.nih.gov/compound/DL-Methionine_sulfoxide 11.48 12.48 166.0533
 0.80 	L-carnitine C7H15NO3 161.105194092  8.97 9.97 162.1126
 0.28 	5-oxo-L-proline C5H7NO3 129.04258728  9.5 10.5 130.0499
 0.32 	|L-histidine| C6H9N3O2 155.069473267  13.18 14.18 156.0767


IndexError: list index out of range

In [6]:
c

{'creation_time': '2015-10-08T11:37:24',
 'description': u'',
 'formula': u'C6H9N3O2',
 'functional_sets': [],
 'head_id': u'cac6a7a3914c454982c247b4bc4d3cd6',
 'inchi': u'InChI=1S/C6H9N3O2/c7-5(6(10)11)1-4-2-8-3-9-4/h2-3,5H,1,7H2,(H,8,9)(H,10,11)/t5-/m0/s1',
 'inchi_key': None,
 'last_modified': '2016-01-23T22:23:18',
 'mono_isotopic_molecular_weight': 155.069473266602,
 'name': u'|L-histidine|',
 'neutralized_2d_inchi': None,
 'neutralized_2d_inchi_key': None,
 'neutralized_inchi': None,
 'neutralized_inchi_key': None,
 'number_components': None,
 'permanent_charge': None,
 'prev_uid': u'origin',
 'reference_xrefs': [],
 'synonyms': u'',
 'unique_id': u'cac6a7a3914c454982c247b4bc4d3cd6',
 'url': u'',
 'username': u'jimmy'}

In [7]:
#     myID.description = 'mz=%5.4f,ppm=%5.4f,RTmin=%5.4f,RTmax=%5.4f,RTpeak=%5.4f'%(mz[i],
#                                                                                  mz_tolerance[i],
#                                                                                  rt_min[i],
#                                                                                  rt_max[i],
#                                                                                  rt_peak[i])
#     print myID.references


all_identifications = []
for i,c in enumerate(myCompounds):
    mzRef = metob.MzReference()

    # take the mz value from the spreadsheet
    mzRef.mz = mz[i]
    
    #TODO: calculate the mz from theoretical adduct and modification if provided.
#     mzRef.mz = c.MonoIso topic_molecular_weight + 1.007276
    mzRef.mz_tolerance = mz_tolerance[i]
    mzRef.mz_tolerance_units = 'ppm'
    mzRef.detected_polarity = 'positive'
#     mzRef.adduct = '[M-H]'
    
    rtRef = metob.RtReference()
    rtRef.rt_units = 'min'
    rtRef.rt_min = rt_min[i]
    rtRef.rt_max = rt_max[i]
    rtRef.rt_peak = rt_peak[i]
    
    myID = metob.CompoundIdentification()
    myID.compound = [c]
    myID.mz_references = [mzRef]
    myID.rt_references = [rtRef]

    all_identifications.append(myID)

myAtlas = metob.Atlas()
#metob.Atlas() has "compound_identifications" and a "name"
# myAtlas.name = '20151130_LS_Negative_Hilic_QExactive_Archetypes'
myAtlas.name = '20151210_SJ_Positive_Hilic_QTOF6550_HMArkin'
myAtlas.compound_identifications = all_identifications
metob.store(myAtlas)

In [8]:
myAtlas

{'compound_identifications': [{'compound': [{'creation_time': '2015-10-08T11:37:24',
 'description': u'',
 'formula': u'C3H7NO2',
 'functional_sets': [],
 'head_id': u'8666a6d45ea14c049923f8e0eb7c005e',
 'inchi': u'InChI=1S/C3H7NO2/c1-2(4)3(5)6/h2H,4H2,1H3,(H,5,6)/t2-/m0/s1',
 'inchi_key': None,
 'last_modified': '2016-01-23T22:23:18',
 'mono_isotopic_molecular_weight': 89.0476760864258,
 'name': u'L-alanine',
 'neutralized_2d_inchi': None,
 'neutralized_2d_inchi_key': None,
 'neutralized_inchi': None,
 'neutralized_inchi_key': None,
 'number_components': None,
 'permanent_charge': None,
 'prev_uid': u'origin',
 'reference_xrefs': [],
 'synonyms': u'',
 'unique_id': u'8666a6d45ea14c049923f8e0eb7c005e',
 'url': u'',
 'username': u'jimmy'}],
 'creation_time': '2016-01-25T09:37:49',
 'description': u'No description',
 'frag_references': [],
 'head_id': u'72736e2dde8f48688703a8aacc3e1b63',
 'identification_grade': None,
 'last_modified': '2016-01-25T09:37:49',
 'mz_references': [{'adduct':

In [9]:
atlases = metob.retrieve('Atlas',name='20151130_LS_Positive_Hilic_QExactive_Archetypes')
a = atlases[0]
a.compound_identifications[0].mz_references

[{'adduct': u'[M+H]+',
  'creation_time': '2015-11-30T14:43:49',
  'description': u'No description',
  'detected_polarity': 'positive',
  'enabled': True,
  'head_id': u'566d6fa9bb6746d3b31122326b2a19de',
  'last_modified': '2016-01-23T22:23:18',
  'lcms_run': None,
  'modification': u'',
  'mz': 90.0549520864258,
  'mz_tolerance': 15.0,
  'mz_tolerance_units': 'ppm',
  'name': u'Untitled',
  'observed_formula': u'',
  'prev_uid': u'origin',
  'ref_type': u'',
  'unique_id': u'566d6fa9bb6746d3b31122326b2a19de',
  'username': u'jimmy'}]

In [10]:
print atlases[0].compound_identifications[0].compound[0].name
print atlases[0].compound_identifications[0].mz_references[0].mz
print atlases[0].compound_identifications[0].rt_references[0].rt_peak
# print atlases[0].compound_identifications[0].references[2].RTpeak
print atlases[0].compound_identifications[0].description.split(',')
print atlases[0].compound_identifications[0].name


L-alanine
90.0549520864
11.02
[u'No description']
Untitled
